In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

from tqdm import tqdm, trange
import tensorflow as tf 
from keras.layers.core import Dense, Dropout, Activation
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Sequential
from keras.layers import LSTM,Bidirectional,GRU
from sklearn.metrics import mean_absolute_error,mean_squared_error

In [ ]:
def root_mean_squared_error(y_true, y_pred):
        return np.sqrt(np.mean(np.square(y_pred - y_true)))

In [ ]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, single_step=False):
  data = []
  labels = []
  
  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size
  
  for i in range(start_index, end_index):
    indices = range(i-history_size, i)
    data.append(dataset[indices])
    
    if single_step:
      labels.append(target[i+target_size])
    else:
      labels.append(target[i:i+target_size])
  
  return np.array(data), np.array(labels)


In [ ]:
##GRU
def gru_model(input_length, input_dim):#input_length所選取天數;input_dim所選取特徵數

    d=0.3
    #return_sequences : True 為維持 (batch ,time_steps ,seq_len) ，連接下一層LSTM 設置 False 將會變成一維
    model= Sequential()
    model.add(GRU(256,input_shape=(input_length, input_dim),return_sequences=True))
    model.add(Dropout(d))
    model.add(Dropout(d))
    model.add(GRU(256,input_shape=(input_length, input_dim),return_sequences=True))
    model.add(Dropout(d))
    model.add(GRU(256,input_shape=(input_length, input_dim),return_sequences=False))
    model.add(Dropout(d))
    
 
    model.add(Dense(1,activation='linear',kernel_initializer="uniform"))#linear / softmax(多分類) / sigmoid(二分法)

    # optimizer = tf.keras.optimizers.Adam(lr=0.00005)
    model.compile(loss='mse',optimizer='adam', metrics=['mean_squared_error'])#loss=mse/categorical_crossentropy
    return model 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/',force_remount=True)
#%cd /content/drive/My-Drive/
# 在線GPU性能检验
!nvidia-smi 
from google.colab import drive
drive.mount('/content/gdrive/',force_remount=True)
#%cd /content/drive/My-Drive/
# 在線GPU性能检验
!nvidia-smi 
raw = pd.read_csv('/content/gdrive/MyDrive/科技部計畫/data/變數預測股價/DayForQuarter.csv',index_col=[0],parse_dates=["Date"])

#read the data
stock = raw 
stock = stock.loc[:,["Close",'Average Confirmation Time',"Block Height",'Difficulty','Miners Revenue (USD)','Unique Addresses Used','Blockchain Size (MB)','Output Value Per Day','Volume','RPI','TOTRESNS','NONREVSL','SP500','VXVCLS','DJIA','NetworkVolume','NBI','XAUUSD']]
amount = len(stock) 
stock = stock['2017-01-01':'2022-07-30']
stock[1619:]

Mounted at /content/gdrive/
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

Mounted at /content/gdrive/
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



,Close,Average Confirmation Time,Block Height,Difficulty,Miners Revenue (USD),Unique Addresses Used,Blockchain Size (MB),Output Value Per Day,Volume,RPI,TOTRESNS,NONREVSL,SP500,VXVCLS,DJIA,NetworkVolume,NBI,XAUUSD
Date,,,,,,,,,,,,,,,,,,
2021-06-29,41564.36,168.03,687456.0,1.990000e+13,20581265.56,379066.0,352241.73,1376609.20,3.790146e+10,17827.22,3848.1,3308.21,4291.80,19.49,34292.29,415.0,5129.75,1761.09
2021-06-30,43790.89,168.03,687456.0,1.990000e+13,20581265.56,567655.0,352241.73,1376609.20,3.405904e+10,17827.22,3848.1,3308.21,4297.50,19.51,34502.51,292.0,5147.93,1769.80
2021-07-01,48116.94,168.03,687456.0,1.990000e+13,22482193.60,567655.0,352608.29,1376609.20,3.783896e+10,17985.18,3943.9,3318.74,4319.94,19.45,34633.53,332.0,5207.87,1776.60
2021-07-02,47711.49,191.55,687456.0,1.990000e+13,22482193.60,567655.0,352608.29,1732809.50,3.872897e+10,17985.18,3943.9,3318.74,4352.34,19.30,34786.35,249.0,5184.26,1786.79
2021-07-03,48199.95,191.55,689472.0,1.990000e+13,22482193.60,583136.0,352608.29,1732809.50,2.438396e+10,17985.18,3943.9,3318.74,4352.34,19.30,34786.35,247.0,5184.26,1786.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-26,19225.70,28.97,741888.0,2.960000e+13,18725062.51,557791.0,413407.51,3655204.28,1.802717e+10,17653.31,3228.4,3501.68,3911.74,28.95,31500.68,145.0,3849.63,1826.18
2022-06-27,19081.00,128.29,741888.0,2.960000e+13,18725062.51,557791.0,413407.51,2980504.09,2.096570e+10,17653.31,3228.4,3501.68,3900.11,28.18,31438.26,245.0,3840.99,1822.73
2022-06-28,19411.00,128.29,741888.0,2.960000e+13,18725062.51,624229.0,413407.51,2980504.09,2.138154e+10,17653.31,3228.4,3501.68,3821.55,29.15,30946.99,240.0,3750.31,1819.69


In [ ]:
# lcd[~(lcd['收盤價-除權息'])]
y = stock["Close"]
x = stock
x.shape

(1981, 18)

In [ ]:
#將資料正規化，讓資料變成0~1之間，讓資料訓練速度更快，且更容易收斂

y= y.values.reshape(-1,1) 
scaler=MinMaxScaler(feature_range=(0,1))
scaler1=MinMaxScaler(feature_range=(0,1))
y=scaler.fit_transform(y)
x=scaler1.fit_transform(x)



In [ ]:
#並將資料分成訓練組50，驗證組25，測試組25
#def multivariate_data(dataset, target, start_index, end_index, history_size,
#                      target_size, single_step=False):
x,y=multivariate_data( x ,y , 0 ,None, 1 , 1 ,single_step=True)
i=1619
x_train,y_train  = x[0:int(i)] , y[0:int(i)]
x_vaild,y_vaild  = x[int(i*2/3):int(i)] , y[int(i*2/3):int(i)]
x_test ,y_test   = x[int(i):-1] , y[int(i):-1]


In [ ]:
trainindex= stock.index[10:len(x_train)+10]
valindex = stock.index[len(x_train)+10:len(x_train)+10+len(x_vaild)]

In [ ]:
print(x_train.shape,y_train.shape,x_vaild.shape,y_vaild.shape,x_test.shape ,y_test.shape )

(1619, 1, 18) (1619, 1) (540, 1, 18) (540, 1) (359, 1, 18) (359, 1)


In [ ]:
##LSTM
def lstm_model(input_length, input_dim):

    d=0.5
    model = Sequential()
    model.add(LSTM(units=80, activation="relu", return_sequences=False, input_shape=(input_length, input_dim)))
    model.add(Dropout(d))  
    model.add(Dense(units=1))

    # optimizer = tf.keras.optimizers.Adam(lr=0.00005)
    model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(lr=0.001), metrics=['mean_squared_error'])#loss=mse/categorical_crossentropy
    return model 

In [ ]:
##build LSTM model
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=70, monitor = 'val_loss')
    ]######## 在訓練組訓練，使用驗證組選取
#EarlyStopping : 在訓練組訓練參數，以驗證組最低為選擇標準，如果300個epochs，沒有改善即停止訓練
filepath="lstm.best.hdf5" #模型儲存路徑
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, mode='min',save_best_only=True)
#選擇val_loss最低的當作最後的模型
call_backlist = [ my_callbacks,checkpoint]
callbacks=call_backlist
lstm = lstm_model(1,18)
historylstm = lstm.fit( x_train, y_train, batch_size=64,shuffle=False , epochs=100,validation_data=(x_vaild,y_vaild),callbacks=call_backlist)




Epoch 1/100


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


20/26 [======================>.......] - ETA: 0s - loss: 0.0047 - mean_squared_error: 0.0047
Epoch 1: val_loss improved from inf to 0.06516, saving model to lstm.best.hdf5
26/26 [==============================] - 4s 30ms/step - loss: 0.0321 - mean_squared_error: 0.0321 - val_loss: 0.0652 - val_mean_squared_error: 0.0652
Epoch 2/100
20/26 [======================>.......] - ETA: 0s - loss: 0.0110 - mean_squared_error: 0.0110
Epoch 2: val_loss improved from 0.06516 to 0.03863, saving model to lstm.best.hdf5
26/26 [==============================] - 0s 10ms/step - loss: 0.0200 - mean_squared_error: 0.0200 - val_loss: 0.0386 - val_mean_squared_error: 0.0386
Epoch 3/100
26/26 [==============================] - ETA: 0s - loss: 0.0203 - mean_squared_error: 0.0203
Epoch 3: val_loss improved from 0.03863 to 0.03386, saving model to lstm.best.hdf5
26/26 [==============================] - 0s 13ms/step - loss: 0.0203 - mean_squared_error: 0.0203 - val_loss: 0.0339 - val_mean_squared_error: 0.0339
Ep

In [ ]:
##LSTM model predict performace
lstm_train  = lstm.predict(x_train)
lstm_val = lstm.predict(x_vaild)
lstm_pre = lstm.predict(x_test)
plt.plot(historylstm.history['loss'])
plt.plot(historylstm.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

pre = lstm.predict(x_train)
pre1=lstm.predict(x_vaild)
fc=np.concatenate((pre,pre1))
yreal=np.concatenate((y_train,y_vaild))
plt.figure(facecolor='white')
pd.Series(fc.reshape(-1)).plot(color='red', label='Predict_LSTM')
pd.Series(yreal.reshape(-1)).plot(color='blue', label='Original_LSTM')

plt.legend()
plt.show()

lstm_pre= scaler.inverse_transform(lstm_pre)
y_test = scaler.inverse_transform(y_test.reshape(-1,1))




plt.figure()
plt.plot(lstm_pre)
plt.plot(y_test)
plt.title('pre')
plt.ylabel('股價')
plt.xlabel('day')
plt.legend(['pre', 'Test'], loc='upper left')
plt.show()
root_mean_squared_error(lstm_pre,y_test) 
from sklearn.metrics import r2_score
r2_score(y_test,lstm_pre)

array([[40934.28 ],
       [44410.805],
       [48423.836],
       [48178.047],
       [48642.027],
       [49272.17 ],
       [51720.805],
       [54787.277],
       [53202.727],
       [53805.57 ],
       [54716.66 ],
       [54303.344],
       [55748.58 ],
       [54491.027],
       [55662.61 ],
       [55447.723],
       [59110.438],
       [58480.797],
       [58939.664],
       [57825.2  ],
       [59688.598],
       [61276.234],
       [60061.51 ],
       [59045.777],
       [59400.203],
       [58208.41 ],
       [59679.15 ],
       [58101.73 ],
       [57411.31 ],
       [59116.15 ],
       [60511.96 ],
       [60313.293],
       [59682.14 ],
       [60054.145],
       [61526.13 ],
       [61585.1  ],
       [60272.23 ],
       [60629.13 ],
       [60172.48 ],
       [61359.977],
       [63372.91 ],
       [63060.07 ],
       [61211.36 ],
       [62331.406],
       [61913.1  ],
       [62320.496],
       [61923.68 ],
       [59906.02 ],
       [57293.656],
       [58865.76 ],
